# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
df = "../output_data/CityDF.csv"
city_data = pd.read_csv(df)
city_data.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Bluff,-46.60,168.33,55.99,66,52,3.00
1,Hermanus,-34.42,19.23,67.55,85,100,19.93
2,Ancud,-41.87,-73.82,55.40,100,90,2.24
3,Torredonjimeno,37.77,-3.96,48.79,81,80,3.76
4,Ushuaia,-54.80,-68.30,44.60,75,40,14.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure(api_key=g_key)


In [20]:
location_data = city_data[['Latitude', 'Longitude']]
humidity_data = city_data['Humidity'].astype(float)
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(location_data, weights=humidity_data, dissipating=False, max_intensity=3, point_radius=.75)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
weather_crit = city_data
temperature = (weather_crit["Temperature"] <=80) & (weather_crit["Temperature"] > 70)
wind_speed = weather_crit["Wind Speed"] < 10
cloudiness = weather_crit["Cloudiness"] == 0

weather_crit = weather_crit[temperature & wind_speed & cloudiness]
weather_crit

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
82,Goundi,9.36,17.37,76.98,16,0,3.87
99,Lakes Entrance,-37.88,147.98,75.42,68,0,7.00
143,Huarmey,-10.07,-78.15,71.53,71,0,1.21
154,San Patricio,19.22,-104.70,75.88,77,0,2.89
189,Pradópolis,-21.36,-48.07,71.60,83,0,4.70
190,Tomatlán,19.93,-105.25,71.85,51,0,2.53
194,Puerto Escondido,15.85,-97.07,70.66,63,0,3.67
257,Oussouye,12.48,-16.55,74.32,94,0,2.24
374,São Filipe,14.90,-24.50,70.68,77,0,4.97
401,Lima,-12.04,-77.03,75.20,78,0,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = weather_crit
hotel_df["Hotel Name"]=""
hotel_df

C:\Users\garrettw13\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
82,Goundi,9.36,17.37,76.98,16,0,3.87,
99,Lakes Entrance,-37.88,147.98,75.42,68,0,7.00,
143,Huarmey,-10.07,-78.15,71.53,71,0,1.21,
154,San Patricio,19.22,-104.70,75.88,77,0,2.89,
189,Pradópolis,-21.36,-48.07,71.60,83,0,4.70,
190,Tomatlán,19.93,-105.25,71.85,51,0,2.53,
194,Puerto Escondido,15.85,-97.07,70.66,63,0,3.67,
257,Oussouye,12.48,-16.55,74.32,94,0,2.24,
374,São Filipe,14.90,-24.50,70.68,77,0,4.97,
401,Lima,-12.04,-77.03,75.20,78,0,6.93,


In [29]:
hotel_name = []
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    latitude = row['Latitude']
    longitude= row['Longitude']
    params["location"]=f"{latitude},{longitude}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    request = requests.get(base_url, params=params)
    
    request = request.json()
    
    try:
        hotel_name.append(request['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
hotel_name

[nan,
 'The Esplanade Resort & Spa',
 'Hotel la siesta',
 'Grand Isla Navidad Resort',
 'Palace Hotel - Pradópolis',
 'Hotel Misión San Miguel',
 'Bungalows Zicatela',
 nan,
 'Hotel Miramar Fogo Brava',
 'Sheraton Lima Hotel & Convention Center',
 nan,
 'Hotel Dawei',
 'Hotel Casa Do Capitao']

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map


# Display Map